$\text{CHURN MODELLING using ANNs(3 layered)}$

Churn modelling via deeplearning (a sub set of machine learning)

!unzip "Machine Learning A-Z (Codes and Datasets)-20240518T093230Z-001.zip" "bankdata.csv"
this won't work here because it is windows and unzip is primarily reserved to linux

In [1]:
%cd Downloads/

C:\Users\LENOVO\Downloads


C:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Workflow
1. Data preprocessing
2. Building ANN
3. Training ANN
4. Making predictions and evaluation metrics

Importing desired libraries

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

1)  1. Importing dataset and checking for discrepancies.
    2. Encoding categorical data for numerical calculations.
    3. Splitting dataset into train, test.
    4. Feature scaling.

In [3]:
df = pd.read_csv('Churn_Modelling.csv')
df.head(3)
df.info()
df.describe()
#this helps us to understand the objects present in the dataset.
#also keeps a track of non-categorical data since .describe method only presents numerical.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Part A -> Importing dataset and checking for discrepancies.

$\text{Data preprocessing}$

In [4]:
X = df.drop('Exited',axis = 1)
y = df['Exited']
#other ways to extract train data and target variable is via .loc and .iloc
#x = df.iloc([:,:-1]).values this drops the last column and selects all the rows.

In [5]:
X.dropna(axis = 0,inplace = True)
X.drop_duplicates()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


Subpart A completed
Usually simple imputer is used to avoid the loss of data and replace missing values but this may cause data leakage and hence not used here.
Part B -> Encoding categorical data

In [6]:
%%time
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
#instantiated an object
X['Gender'] = enc.fit_transform(X['Gender'])
#could also be achieved via map fucntion or pd.get_dummies selected this since it has less verbose code

CPU times: total: 516 ms
Wall time: 2.89 s


In [7]:
%%time
def gender(sex):
    if sex == 'Male':
        return 1
    else:
        return 0

np.asarray(list(map(gender,X['Gender'])))

CPU times: total: 0 ns
Wall time: 3.37 ms


array([0, 0, 0, ..., 0, 0, 0])

Despite droping the target variable, duplicates and nan values still to process the data and create an ANN relevant features should be provided as input to reduce the computation hence removing Rownumber,Customerid,Surname has a valid call since it can cause overfitting of data 

In [8]:
X = X.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [9]:
#applying one hot encoding on geography column may help since a crisis in a country may have caused them to withdraw their  money from the bank
X.iloc[:,1].nunique()

3

In [10]:
#hence 3 distinct countries exist in the dataset in geography feature
X['Geography'] = enc.fit_transform(X['Geography'])

SubPartB completed
One hot encoding is also used sometimes via Column transformer but it may distort the original feature space hence label encoder is beign used.

Subpart C -> splitting data into train and test

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 101)

SubPartC completed

Subpart D -> Feature scaling (is the most required)

In [12]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_train = ss.transform(X_train)

C:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


$\text{Building the ANN}$

The ANNs are group of sequential layers hence we have to instantiate an object of type sequential which would create a sequential graph and compute the output, computational and sequential graphs are different!!.

1. Initializing an ANN
2. Adding the input layer and the first hidden layer
3. Adding the second hidden layer
4. Adding the output layer

Initializing an ANN

In [13]:
ann = tf.keras.models.Sequential()
#since now keras is integrated into tensorflow itself

Adding the the first hidden layer (shallow hidden layer NN created )
Also units = 6 denote there are 6 neurons in the layer, these are hyperparameters since these are not updated during the process

Adding the input layer is implicit and no need for using keras.

In [14]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

Adding the the second hidden layer 

In [15]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

Adding the the output layer - the units has to be replaced and activation to be replaced with sigmoid to get the probability and if we had a non binary target variable then we would require same number of neurons at last stage since 1 hot encoding would create a 001...0 kind of object 

In [16]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

$\text{Training ANN}$

1. Compiling ANN -> optimizer, loss, metircs(evaluation measure)

In [18]:
#optimizer is sgd here adam optimizer algorithm, loss can be RMSE and metrics is accuracy
ann.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])
#for binary variable binary cross entropy is used and for nonbinary categorical cross entropy is used and activation is softmax

2. Train the ANN on training set

In [19]:
ann.fit(X_train,y_train,batch_size = 16,epochs = 100)

Epoch 1/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7817 - loss: 0.5481
Epoch 2/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8187 - loss: 0.4360
Epoch 3/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8168 - loss: 0.4329
Epoch 4/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8229 - loss: 0.4153
Epoch 5/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8203 - loss: 0.4214
Epoch 6/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8143 - loss: 0.4256
Epoch 7/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8240 - loss: 0.4201
Epoch 8/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8175 - loss: 0.4268
Epoch 9/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8299 - loss: 0.4047
Epoch 10/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8218 - loss: 0.4132
Epoch 11/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8192 - loss: 0.4280
Epoch 12/100
500/500 ━━━━━━━━━━━━━━━━━━━━

Making predictions usign an observation

In [35]:
x = [1,0,0,40,3,60000,2,0,1,50000]

In [36]:
#beware that we are using batch size hence it is important to have tensor object!!!  
ann.predict(ss.transform([x]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


C:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[0.32327875]], dtype=float32)

In [37]:
y_pred = ann.predict(X_test)
y_pred = (y_pred>0.5)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 894us/step


In [38]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
print(cm)

[[941 636]
 [309 114]]


Instead of hard coding batch size and other hyper parametrs we can also create a pipeline for determining the best values.